In [1]:
'''This repository contains a detailed implementation of the Reinforcement Learning Enviroment class'''
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from typing import Any, Callable, Dict, List, Tuple, Union, Optional
from functools import wraps
import os
import random
from abc import ABC, abstractmethod
from collections import deque, namedtuple
import scipy.integrate as integrate
import scipy.optimize as optimize
T.Tensor.ndim = property(lambda self: len(self.shape))

In [ ]:
N = 50    # number of time steps
t_f = 5.0  # final time (s)
dt = t_f / (N - 1)
x_final= np.array([2.0, 0.0, 2.0, 0.0])
x_initial= np.array([0.0, 0.0, 0.0, 0.0])

def dynamics(State, _, ControlForce):
  dxPosition, dyPosition = State[1], State[3]
  Position= T.tensor([State[0], State[2]])
  dxVelocity, dyVelocity = ((PositiveCharge.Charge* TestElectricField(Position))+ControlForce)/PositiveCharge.Mass

  return np.array([dxPosition, dxVelocity, dyPosition, dyVelocity])

def objective(z):
    px, _, py, _, ux, uy = z.reshape(6, N)
    Pos= T.transpose(T.cat([T.tensor([px]), T.tensor([py])], 0), 0, 1)
    EField= [T.norm(TestElectricField(i)) for i in Pos]
    return sum(EField)

def DynamicsConstraint(z):
  px, vx, py, vy, ux, uy = z.reshape(6, N)
  x= np.vstack((px, vx, py, vy)).transpose()
  u= np.vstack((ux, uy)).transpose()
  ceq = []

  for k in range(N-1):
    f_k = dynamics(x[k], 1.0, u[k])
    f_kp1 = dynamics(x[k+1], 1.0, u[k+1])
    ceq.append((x[k+1] - x[k]) - (dt * (f_k + f_kp1))/2)
  return np.array(ceq).transpose().flatten()

def BoundaryConstraint(z):
  px, vx, py, vy, _, _ = z.reshape(6, N)
  xi= np.vstack((px, vx, py, vy)).transpose()[0]
  xf= np.vstack((px, vx, py, vy)).transpose()[-1]
  x= np.concatenate((xi, xf), axis=None)
  x_bounds= np.concatenate((x_initial, x_final), axis=None)
  return x-x_bounds

InitialGuess = np.zeros(6*N)
InitialGuess[:N] = np.linspace(x_initial[0], x_final[0], N)  # xPosition Initial Guess
InitialGuess[N:2*N] = np.linspace(x_initial[0], x_final[1], N)  # xVelocity Initial Guess
InitialGuess[2*N:3*N] = np.linspace(x_initial[0], x_final[2], N)  # yPosition Initial Guess
InitialGuess[3*N:4*N] = np.linspace(x_initial[0], x_final[3], N)  # yVelocity Initial Guess

res = optimize.minimize(objective, InitialGuess, method='SLSQP', constraints=({'type': 'eq', 'fun': DynamicsConstraint}, 
                                                                              {'type': 'eq', 'fun': BoundaryConstraint}))

t = np.linspace(0, t_f, N)
px, vx, py, vy, ux, uy = res.x.reshape(6, N)
plt.plot(px, py)
plt.xlim(-5,5)
plt.ylim(-5,5)
plt.grid(True)
plt.title('Optimal Path')
plt.xlabel('x-Axis(m)')
plt.ylabel('y-Axis(m)')
plt.show()
